# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [14]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

gmaps.configure(api_key=g_key)


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [15]:

weather_csv_file = "../output_data/cities.csv"

weather_df = pd.read_csv(weather_csv_file)


weather_df.head()

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,ostrovnoy,2,RU,1558378754,72,68.05,39.51,37.50,7.16
1,1,mahebourg,75,MU,1558378503,74,-20.41,57.70,78.80,11.41
2,2,qaanaaq,25,GL,1558378755,73,77.48,-69.36,22.20,2.37
3,3,zhuhai,0,CN,1558378755,39,40.71,112.04,44.60,4.47
4,4,cape town,20,ZA,1558378755,76,-33.93,18.42,55.99,8.05


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [16]:
locations = weather_df[["Lat", "Lng"]]
# locations

humidity = weather_df["Humidity"].astype(float)

# humidity

# figure_layout = {
#     'width' : '1000px',
#     'height':'400px',
#     'border':'1px solid black',
#     'padding':'1px',
#     'margin':'0 auto 0 auto'
# }

# fig = gmaps.figure(layout=figure_layout)

fig = gmaps.figure()

heatmap_layer = gmaps.heatmap_layer(locations, weights=humidity,
                                   dissipating=False, max_intensity=5,
                                   point_radius=1)

fig.add_layer(heatmap_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [18]:
# Narrow down the DataFrame to find your ideal weather condition. For example:
# A max temperature lower than 80 degrees but higher than 70.
# Wind speed less than 10 mph.
# Zero cloudiness.
# Drop any rows that don't contain all three conditions. 
# You want to be sure the weather is ideal

new_types_df = pd.DataFrame(weather_df, columns = ["City", "Max Temp", "Wind Speed", "Cloudiness"])

max_temp = (new_types_df["Max Temp"] <= 80) & (new_types_df["Max Temp"] > 70)
wind_speed = new_types_df["Wind Speed"] < 10
cloudiness = new_types_df["Cloudiness"] == 0

# new_types_df[max_temp & wind_speed & cloudiness]
new_types_df[max_temp & wind_speed & cloudiness]

,City,Max Temp,Wind Speed,Cloudiness
7,opuwo,72.06,5.44,0
70,ponta do sol,73.68,5.32,0
88,cidreira,76.56,5.50,0
138,morondava,79.08,9.06,0
139,vaini,78.90,1.59,0
176,nador,75.20,6.93,0
297,mogok,72.24,2.77,0
444,birjand,71.60,9.17,0


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [19]:

hotel_df = pd.DataFrame(weather_df, columns=["City", "Country", "Lat", "Lng"])

hotel_df["Hotel Name"] = ""
# hotel_df["Coordinates"]=""
hotel_df

,City,Country,Lat,Lng,Hotel Name
0,ostrovnoy,RU,68.05,39.51,
1,mahebourg,MU,-20.41,57.70,
2,qaanaaq,GL,77.48,-69.36,
3,zhuhai,CN,40.71,112.04,
4,cape town,ZA,-33.93,18.42,
...,...,...,...,...,...
543,yabrud,SY,33.97,36.66,
544,paraiso,MX,24.01,-104.61,
545,veraval,FR,49.65,0.71,
546,novyy urgal,RU,51.07,132.56,


In [20]:

# coordinates_lat_lng = []

# target_city = hotel_df["City"]
# # target_city

# target_type = "hotel"

# hotel_df["Coordinates"] = hotel_df["Lat"] + "," + hotel_df["Lng"]

# radius = 5000

# params = {
#     "location:" : coordinates_lat_lng,
#     "types" : target_type,
#     "radius" : radius, 
#     "key" : g_key
# }

# base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# print("A HOTEL WITHIN A 5000 METER RADIUS")

# response = requests.get(base_url, params)
# # print(response.url)

# hotel_distance = response.json()
# print(json.dumps(hotel_distance, indent=4, sort_keys=True))

target_coordinates = f"{hotel_df['Lat'][0]},{hotel_df['Lng'][0]}"
target_search = "hotel"
target_radius = 5000
target_type = "hotel"

# set up a parameters dictionary
params = {
    "location": target_coordinates,
    "keyword": target_search,
    "radius": target_radius,
    "type": target_type,
    "key": g_key
}

# base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# run a request using our params dictionary
response = requests.get(base_url, params=params)

In [22]:
# base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
# params = {
#     "location": (hotel_df["Lat"], hotel_df["Lng"]),
#     "rankby": "distance",
#     "type" : "hotel", 
#     "key" : g_key
# }

# # params
from pprint import pprint
pprint(response.json(), depth=1)


{'html_attributions': [], 'results': [], 'status': 'ZERO_RESULTS'}


In [23]:
hotel_df.iloc[0][0]

'ostrovnoy'

In [24]:

#creates a list to capture hotel_names to insert into dataframe
hotel_name = []

# params dictionary to update each iteration
params = {
    "radius": 5000,
    "types": "hotel",
    "keyword": "hotel",
    "key": g_key
}

# taken from Airpost_Ratings Example done in class
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    name_address = requests.get(base_url, params=params)
    
    # convert to json
    name_address = name_address.json()
    # print(json.dumps(name_address, indent=4, sort_keys=True))
    try:
#         hotel_df.iloc[index, 4] = name_address['results'][0]['name']
        hotel_name.append(name_address['results'][0]['name'])
    except IndexError:
        hotel_name.append(np.nan)

In [25]:
hotel_name

[nan,
 'Shandrani Beachcomber Resort & Spa',
 'Qaanaaq Hotel',
 nan,
 'Mojo Hotel',
 'Simplicité Marquises',
 'Oaks Hotel Motel',
 'Opuwo Country Lodge',
 'Umka',
 'Kottedzh V Sharobykakh',
 'Hotel Astoria',
 'Enjoy Chiloé. Hotel De La Isla',
 'The Edgewater Resort and Spa',
 'Saskylakh Aeroport',
 'Hotel Gobernador',
 'Renaissance Albany Hotel',
 'Gostinitsa Viliuisk',
 'Las Lengas Hotel',
 'Laguna Beach Hotel & Spa',
 'Отель Купеческий',
 'Milky Resort',
 'Esplanade Hotel',
 'Domaine de Rochevilaine',
 'Salinas do Maragogi All Inclusive Resort',
 nan,
 'โรงแรมกันตังปาร์ค(Kantang Park Hotel)',
 'Anand Hotel',
 'Hotel Plaza',
 "Pension Maro'i",
 'Old Colony Inn',
 'Onze Rust Guest House',
 'Shangri-La Hotel Qingdao',
 'Pier Side Landing',
 nan,
 nan,
 'Long House Hotel Bethel',
 'Holiday Inn Express & Suites Clinton',
 'Huset ved havet',
 nan,
 nan,
 'Misty Waves Boutique Hotel',
 "Best Western Plus St. John's Airport Hotel and Suites",
 'The Explorer Hotel',
 nan,
 'Hotel Solymar Gala

In [27]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [28]:
# Add marker layer ontop of heat map

markers = gmaps.marker_layer(locations)
fig.add_layer(markers)
fig

# Display Map

# Display Map

Figure(layout=FigureLayout(height='420px'))